In [ ]:
# basic python system/path library
import os
import glob
import shutil
from collections import defaultdict, OrderedDict
import time
from configparser import ConfigParser

# numerical libraries
import numpy as np
import pandas as pd

# binary file libraries
import pyarrow
import openpyxl

# HTTP library
import requests

# Compression libraries
import zipfile
import tarfile
import gzip

# plotting libraries
import matplotlib.pyplot as plt
plt.rc('axes', titlesize=30)
plt.rc('axes', labelsize=20)
import seaborn as sns
import contextily as cx

# database libraries
import pyodbc

# Geographical processing data
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point

## Funciones

In [ ]:
def unzip(path_zip, unzip_dir, password=None):
    """Descomprime el fichero de entrada en un directorio de 
    salida.

    Args:
        path_zip (atr): Ruta al fichero comprimido en ZIP.
        unzip_dir (str): Directorio donde se descomprimirá el fichero.
        password (str, optional): Constraseña para descomprimir el fichero de entrada. Defaults to None.

    Returns:
        int: 0, si el proceso se ha desarrollado sin errores.
             1, en caso contrario.
    """
    file_zip = zipfile.ZipFile(path_zip, "r")
    try:
        print(f'"{path_zip}" content:')
        print(file_zip.namelist())
        file_zip.extractall(pwd=password, path=unzip_dir)
    except Exception as e:
        print(f'ERROR: Se ha producido un error al descomprimir el fichero "{file_zip}".')
        print(str(e))
        return 1

    file_zip.close()
    
    return 0

In [ ]:
def get_url(url, download_dir):
    """Descarga el fichero apuntado por 'url' en el directorio 'download_dir'.
    
    Args:
        url (str): Fichero apuntado por la URL.
        download_dir (str): Directorio donde se almacenará el fichero descargado.

    Returns:
        int: Número de caracteres descargados.
             -1, en caso de error.
    """
    
    
    prefix, name = os.path.split(url)
    
    if not os.path.exists(download_dir):
        try:
            os.makedirs(download_dir, exist_ok=True)
        except IOError as e:
            print(f'ERROR: No se pudo generar el directorio de salida "{download_dir}".')
            print(str(e))
            return -1
    
    # ruta de descarga del fichero apuntado por la url
    download_path = os.path.join(download_dir, name)
    
    # obteniendo datos de la url mediante el método GET
    r = requests.get(url)
    
    # Escribiendo el fichero descargado en la ruta de descarga
    with open(download_path, 'wb') as f:
        f.write(r.content)
    
    return len(r.content)

In [ ]:
def read_db_piezometry(path):
    """Lee la información contenida en el fichero MS Access que viene dado por el parámetro 'path'.

    Args:
        path (str): Ruta en disco al fichero MS Access.

    Returns:
        tuple: (pd.DataFrame piezometros, pd.DataFrame niveles)
    """

    # Conexión a MS Access
    path = path.replace(r'\\', '/')
    print(f'path={path}')    
    cnxn = pyodbc.connect(r'DRIVER={};DBQ={};UID="";PWD="";'.format('Microsoft Access Driver (*.mdb, *.accdb)', path)) 
    
    
    # Consulta sobre los datos de Piezometros a MS Access
    query_piezometros = pd.read_sql('SELECT * FROM 01_Piezometros', con=cnxn)
    
    # Paso a dataframe
    df_piezometros = pd.DataFrame(query_piezometros)
    
    # Consulta sobre las medias de piezometría a MS Access
    query_niveles = pd.read_sql(sql='SELECT * FROM 02_Niveles', con=cnxn, parse_dates={'FechaP':'%Y-%m-%d'})
    
    # Paso a dataframe
    df_niveles = pd.DataFrame(query_niveles)
   
    return df_piezometros, df_niveles

In [ ]:
def media_ponderada_por_distancia(distances, df_measures):
    """Determina el valor de la media ponderada por el inverso de la distancia de
    las medidas dadas por el parámetro df_measures.

    Args:
        distances (np.array): Array de distancias entre puntos considerados.
        df_measures (pd.Serie): Serie de medidas tomadas.

    Returns:
        np.array: Array de medidas promedio de las magnitudes pasadas 
            ponderadas por el inverso de la distancia.
    """

    weights = 1 / distances
    
    return np.dot(weights, df_measures.to_numpy().T) / weights.sum()

In [ ]:
def get_precipitations(gdf_location, gdf_nodes_grid, df_measures, mode='closest', radius_m=5000.0, field_date='fecha'):
    """Devuelve un dataframe con fechas como índices y medidas de precipitación calculadas según el 
    algoritmo dado por el parámetro 'mode'.

    Args:
        gdf_location (gpd.GeoDataFrame): GeoDataFrame con información sobre las ubicaciones de muestreo.
        gdf_nodes_grid (gdf.GeoDataFrame): GeoDataFrame con información sobre los puntos de los que 
            se dispone de medidas de precipitación.
        df_measures (pd.DataFrame): DataFrame con las medidas de precipitación, para cada punto 
        referenciado en 'gdf_nodes_grid'.
        mode (str, optional): Algoritmo empleado para el cálculo de la precipitación que se 
            asociará a cada punto dado en 'gdf_location'. Hay dos posibilidades: 
                - 'closest', asocia la precipitación del punto más cercano.
                - cualquier otro valor, que use el algoritmo del promedio de la precipitación
                    ponderado por el inverso de la distancia para los puntos con medida de 
                    precipitación más cercanos.
            Defaults to 'closest'.
        radius_m (float, optional): Distancia máxima (en metros) a los puntos considerados como 
            próximos. Defaults to 5000.
        field_date (str, optional): Nombre del campo que contiene la fecha de las mediciones de 
            precipitacion en 'df_measures'. Defaults to 'fecha'.

    Returns:
        tuple: (precipitaciones, asociaciones)
            donde
                - precipitaciones (dict), diccionario cuyas claves son los valores de los índices
                    del GeoDataFrame 'gdf_location', y cuyo valor es la precipitación estimada según el 
                    algoritmo seleccionado.
                - asociaciones (dict), diccionario que contiene la asociación realizada entre 
                    puntos de muestreo y de precipitación. Sus claves son: 
                        index_loc, index_grid y distance
                    donde 
                        - 'index_loc' (list), son loas valores de los índices del GeoDataFrame 
                            'gdf_location' que identifican cada punto de muestreo,
                        - 'index_grid' (list), es una lista con los nodos de precipitación que se relacionan
                            con cada punto de muestreo (este valor puede ser una lista de listas), y
                        - distance (list), es una lista con las distancias (en metros) de cada punto de 
                            interés a los nodos de precipitación con los que se relaciona (este valor 
                            puede ser una lista de listas).
    """

    # Diccionario de listas donde almacenaremos los resultados de puntos cercanos
    pairs = defaultdict(list)
    measure = OrderedDict()
    measure['fecha']= df_measures.loc[:, field_date].values

    # creo el buffer alrededor de cada localización
    gdf_location["buffered"] = gdf_location.buffer(radius_m)
    
    # para cada localización de medida...
    for ind in gdf_location.index:
        location = gdf_location.loc[[ind]]
        # print(f'location = {location}')
        inside = gdf_nodes_grid.geometry.within(location.buffered.values[0]) # boolean serie
        
        if inside.to_numpy().sum() == 0: # not grid points in location buffers
            continue
        
        gdf_closer_nodes = gdf_nodes_grid[inside]
    
        # Computing distances
        distances = gdf_closer_nodes.geometry.distance(location.geometry.values[0])

        pairs['index_loc'].append(ind)
        if mode.lower() == 'closest':
            # Tomamos el punto de gdf_nodes_grid más cercano dentro del buffer
            index_min_dist = distances.argmin()
            closest_index_grid = gdf_closer_nodes.index[index_min_dist]
            pairs['index_grid'].append(closest_index_grid)
            pairs['distance'].append(distances.values.min())
            measure[ind] = df_measures.loc[:, closest_index_grid].to_numpy()
        else:
            pairs['index_grid'].append(gdf_closer_nodes.index)
            pairs['distance'].append(distances.values)
            
            # Tomamos los puntos de gdf_nodes_grid dentro del buffer (media ponderada por distancia)
            measure[ind] = media_ponderada_por_distancia(distances, df_measures.loc[:, gdf_closer_nodes.index])
        
    return measure, pairs

In [ ]:
def get_piezometry(gdf_location, gdf_piezo, df_measures, radius_m=5000.0, field_date='fecha'):
    """Devuelve un dataframe con fechas como índices y medidas. Para cada ubicación de
    'gdf_location', le asocia la serie temporal más larga de medidas de piezometría
    dentro de las disponibles en un radio de 'radius_m' alrededor de dicha ubicación.
    
    Args:
        gdf_location (gpd.GeoDataFrame): GeoDataFrame con información sobre las ubicaciones de muestreo.
        gdf_piezo (gdf.GeoDataFrame): GeoDataFrame con información sobre los puntos de los que 
            se dispone de medidas de piezometría.
        df_measures (pd.DataFrame): DataFrame con las medidas de precipitación, para cada punto 
            referenciado en 'gdf_piezo'.
        radius_m (float, optional): Distancia máxima (en metros) a los puntos considerados como 
            próximos. Defaults to 5000.0.
        field_date (str, optional): Nombre del campo que contiene la fecha de las mediciones de 
            piezometría en 'df_measures'. Defaults to 'fecha'.

    Returns:
        tuple: (piezometrías, asociaciones)
            donde
                - piezometrías (dict), es un diccionario cuyas claves son los valores de los índices
                    del GeoDataFrame 'gdf_location', y cuyo valor es la piezometría estimada según el 
                    algoritmo de serie de medidas piezométricas más larga de entre las ubicaciones 
                    de los piezómetros cercanos.
                - asociaciones (dict), diccionario de listas que contiene la asociación realizada entre 
                    puntos de muestreo y de precipitación. Sus claves son: 
                        index_loc, index_piezo y distance
                    donde 
                        - 'index_loc' (list), son loas valores de los índices del GeoDataFrame 
                            'gdf_location' que identifican cada punto de muestreo,
                        - 'index_piezo' (list), es una lista con los puntos de medida piezométrica que 
                            se relacionan con cada punto de muestreo, y
                        - distance (list), es una lista con las distancias (en metros) de cada punto de 
                            interés a los puntos con medidas piezométricas con los que se relaciona.
    """
    
    # Diccionario de listas donde almacenaremos los resultados de puntos cercanos
    pairs = defaultdict(list)
    measure = OrderedDict()

    # creo el buffer alrededor de cada localización
    gdf_location["buffered"] = gdf_location.buffer(radius_m)
    
    # para cada localización de medida...
    for ind in gdf_location.index:
        location = gdf_location.iloc[[ind]]
        inside = gdf_piezo.geometry.within(location.buffered.values[0]) # boolean serie
        
        if inside.sum() == 0: # not grid points in location buffers
            continue
        
        gdf_close_nodes = gdf_piezo[inside.values]
    
        # Computing distances
        distances = gdf_piezo.geometry.distance(location.geometry.values[0])
        # print(f'Nodos más cercanos = {gdf_close_nodes.index}')
        #print(f'Distancias de los nodos más cercanos = {distances}')
        #break    
        # Eligiendo la serie más larga de las disponibles en el buffer de 'radius_m' metros
        better_index = None
        lon_serie = -1
        for i in gdf_close_nodes.index:
            n_measures = df_measures.loc[i,'fecha'].values.size
            if better_index is None or lon_serie < n_measures:
                better_index = i
                lon_serie = n_measures
        # Tomamos la serie más larga de medidas piezometricas
        measure[ind] = df_measures.loc[better_index,['fecha', 'Cota_NP_msnm']]
        
        pairs['index_piezo'].append(better_index)
        pairs['index_loc'].append(ind)
        pairs['distance'].append(distances[better_index])
            
        
    return measure, pairs

## Parámetros del notebook

In [ ]:
# Lectura del fichero de configuración
config = ConfigParser(inline_comment_prefixes="#")
config.read("config.ini" )

In [ ]:
# List all contents
print("List all contents")
for section in config.sections():
    for option in config.options(section):
        print(f"{section}: {option} = {config.get(section, option)} (type = {type(config.get(section, option))})")

**Main code**

## 1. Puntos de muestreo

Es un fichero en formato CSV que contiene información de objetos puntuales. Los campos están separados por ';'

In [ ]:
locations_file = config.get('paths', 'locations_file')
df_loc = pd.read_csv(locations_file, sep=';', decimal=',')

Establezco como índice del *DataFrame* la columna dada por el campo 'location_id' en el fichero *config.ini*.

In [ ]:
id_loc = config.get('fields', 'location_id')
df_loc = df_loc.set_index(id_loc, drop=False)
print(f'Campo de índice del DataFrame de localizaciones de muestreo = {id_loc}')

¿Son únicas las claves dadas por el campo *location_label* en el fichero *config.ini*?

In [ ]:
label = config.get('fields', 'location_label')
num_locatizaciones = len(df_loc.index)
num_distinct_labels = np.unique(df_loc.loc[:, label].to_numpy()).size
if num_locatizaciones != num_distinct_labels:
    print(f'Las etiquetas se repiten. Tenemos un problema con este campo.')
    u, c = np.unique(df_loc.loc[:, label].to_numpy(), return_counts=True)
    dup_el = u[c > 1]
    dup_count = c[c > 1]
    print(f'Hay {dup_el.size} etiquetas repetidas.')
    for e, n in zip(dup_el, dup_count):
        print(f'"{e}" -> {n} veces repetido')
        
    df_dup = df_loc[df_loc['PS_ID'] == dup_el[0]]
    print('*' * 50)
    print(f'PS_ID repetido = {dup_el[0]}')
    print(df_dup.loc[:, ['X', 'Y', 'Lon_ETRS89', 'Lat_ETRS89']])

Información sobre el objeto GeoDataFrame generado tras la lectura del fichero.

In [ ]:
print(df_loc.info())

Si existe un campo llamado *GEOMETRY*, lo elimino

In [ ]:
if 'GEOMETRY' in df_loc.columns:
    df_loc = df_loc.drop('GEOMETRY', axis=1)

Echamos un vistazo al DataFrame.

In [ ]:
df_loc.head()

Queremos toda información en el sistema de referencia proyectado, dado por el parámetro 'projected' del fichero *config.ini*. Comprobamos entonces los parámetros que determinan los nombres de los campos que representan la longitud (*location_lon_proj*) y latitud (*location_lat_proj*) en coordenadas proyectadas y creamos el *GeoDataFrame* correspondiente. De no estar definidas esas coordenadas, usamos las dadas en el sistema de referencia geográfico ((*location_lon_geo*. *location_lat_geo*)) para generar el *GeoDataFrame* y reproyectamos al sistema de referencia proyectado dado.

In [ ]:
gdf_loc = None
if len(config.get('fields', 'location_lon_proj')) > 0:
    print(f"Campos de coordenadas proyectadas de entrada: ({config.get('fields', 'location_lon_proj')}, {config.get('fields', 'location_lat_proj')})")
    geometry = gpd.points_from_xy(df_loc.loc[:,config.get('fields', 'location_lon_proj')], \
                                 df_loc.loc[:,config.get('fields', 'location_lat_proj')])
    gdf_loc = gpd.GeoDataFrame(df_loc, crs=config.get('crs', 'location_projected'), geometry=geometry)
else:
    print(f"Campos de coordenadas geográficas de entrada: ({config.get('fields', 'location_lon_geo')}, {config.get('fields', 'location_lat_geo')})")
    geometry = gpd.points_from_xy(df_loc.loc[:,config.get('fields', 'location_lon_geo')], \
                              df_loc.loc[:,config.get('fields', 'location_lat_geo')])
    gdf_loc = gpd.GeoDataFrame(df_loc, crs=config.get('crs', 'location_geographic'), geometry=geometry)

# Reproyectamos la información a un sistema de coordenadas proyectadas común a todos los conjuntos 
# de datos de entrada del notebook: queremos trabajar con unidades de metros (m)
gdf_loc = gdf_loc.to_crs(config.get('crs', 'final_projected'))

Muestro de las primeras 5 filas del objeto *GeoDataFrame*. Ahora, se ha creado una nueva columna **geometry** que define cada objeto representado en los registros del fichero (puntos). Vienen dados en el sistema de referencia proyectado.

In [ ]:
print(gdf_loc.head())

Genero una gráfica con las ubicaciones del fichero.

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_loc.plot(color='cyan', edgecolor='blue', alpha=0.1, marker='o', label='Ubicación de muestreo')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='upper left', prop={'size':20}, markerscale=3)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})')
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})')
plt.title('Localizaciones de los puntos de muestreo')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
if not os.path.exists(output_dir):
    try:
        os.makedirs(output_dir)
    except IOError:
        print(f'ERROR: No se pudo crear el directorio de salida "{output_dir}".')
        print('Genérelo de forma manual y vuelva a ejecutar esta celda.')
plot_out = os.path.join(output_dir, 'localizaciones_de_muestreo.jpg')
print(f'Output plot path = "{plot_out}"')
plt.savefig(plot_out, dpi=300)

plt.show()

## 2. Precipitationes

La medida de las precipitaciones se obtiene a partir de la URL proporcionada a través de la
variable *precipitations_url*, en el fichero *config.ini*. Por defecto, tiene asignada la dirección web de un fichero empaquetado y comprimido en formato *.tar.gz*.

Tras descomprimir y desempaquetar pordemos acceder al conteido. Básicamente se trata de: 

- Un fichero con información detallada al respecto del contenido del paquete descargado (**README.txt**).
- las ubicaciones de los puntos de la malla distribuida de forma uniforme por el territorio nacional en cuadrículas de 5x5 km (**maestro_red_hr_SPAIN.txt**), y 
- las medidas de precipitación interpolada en esos nodos (**pcp_red_SPAIN_1951-2020.txt**). La resolución temporal es diaria. Ha sido registrada en el periodo de 1951 a 2020. Se espera que estos datos se vayan actualizando en la web de la AEMET.


###  2.1. Carga de las ubicaciones de los nodos de la malla de precipitación

In [ ]:
precipitations_url = config.get('urls', 'precipitations_url')
data_dir = config.get('directories', 'data_dir')

root, name_prec = os.path.split(precipitations_url)
precip_path = os.path.abspath(os.path.join(data_dir, name_prec))

print(f"Fichero de precipitaciones: '{precip_path}'")

Si no se ha hecho ya, descargamos y descomprimimos el fichero de precipitaciones.

In [ ]:
res_get = 1

if not os.path.exists(precip_path):
    print(f"Descargando: '{precipitations_url}'. Por favor, espere.")
    res_ges = get_url(precipitations_url, data_dir)

if res_get > 0:
    # descomprimo el fichero
    precip_compress = tarfile.open(precip_path)
    precip_compress.extractall(data_dir)
    precip_compress.close()
    
# gunzip fichero de datos
patron = os.path.abspath(os.path.join(data_dir, '*.txt.gz'))
print(f'Buscando ficheros con el patron: {patron}')
precip_gz = glob.glob(patron)
precip_path = None
if len(precip_gz):
    print(f'Fichero a desempaquetar: {precip_gz[0]}')
    precip_path = precip_gz[0][:-3]
    if not os.path.exists(precip_path):
        with gzip.open(precip_gz[0], 'rb') as f_in:
            with open(precip_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

El fichero maestro es el que contiene la ubicación de los nodos de la malla de precipitaciones.

In [ ]:
master_path = os.path.abspath(os.path.join(data_dir, 'maestro_red_hr_SPAIN.txt'))
df_master_precip = pd.read_csv(master_path, header=None, delim_whitespace=True)

Las coordenadas de los nodos de precipitación interpolada vienen dada en el sistema de referencia geográfico.

In [ ]:
# Establecemos los nombres de columnas del fichero, ya que inicialmente no tiene
df_master_precip.columns = ['ID', config.get('fields', 'precip_lon_geo'), config.get('fields', 'precip_lat_geo'), 'height']

In [ ]:
# Establecemos como índice del Dataframe creado al campo 'ID'
df_master_precip = df_master_precip.set_index('ID', drop=False)

# Mostramos el *DataFrame*
print(df_master_precip)

*precip_lon_geo* y *precip_lat_geo* vienen dados en coordenadas geográficas. Generamos el correspondiente *GeoDataFrame* a partir de esas columnas.

In [ ]:
geometry = gpd.points_from_xy(df_master_precip.longitude, df_master_precip.latitude)
gdf_master_precip = gpd.GeoDataFrame(df_master_precip, crs=config.get('crs', 'precip_geographic'), geometry=geometry)

El índice del *GeoDataFrame* es el campo que hemos llamado 'ID' (el mismo que el del *DataFrame* que le da su origen).

In [ ]:
print(gdf_master_precip.info())

In [ ]:
print(gdf_master_precip.head())

El sistema de referencia geográfico (parámetro *precip_geograhic* del fichero *config.ini*) viene dado en grados. Tengo que cambiar a otro sistema de referencia proyectado (parámetro *final_projected* del fichero *config.ini*) para obtener medidas de ubicación y distancias en metros (m). Al final se plotean de nuevo los puntos para comprobar que la proyección se ha hecho correctamente.

In [ ]:
crs_proyectado = config.get('crs', 'final_projected')
gdf_master_precip = gdf_master_precip.to_crs(crs_proyectado)

print(f'gdf_loc.crs = {gdf_loc.crs}')
print(f'gdf_master_precip.crs = {gdf_master_precip.crs}')

Muestro la malla y los puntos de muestreo sobre los que vamos a trabajar.

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_loc.plot(color='blue', alpha=0.3, marker='.', markersize=10, label='Ubicación de muestreo')
gdf_master_precip.plot(ax=ax, color='red', edgecolor='red', marker='.', markersize=2, label='Nodo de precipitación')

cx.add_basemap(ax, crs=gdf_loc.crs.to_string(), alpha=0.5)
plt.legend(loc='lower right', prop={'size':20}, markerscale=6)
plt.xlabel(f"Longitud ({config.get('crs', 'final_projected')}", fontsize=17)
plt.ylabel(f"Latitud ({config.get('crs', 'final_projected')})", fontsize=17)
plt.title('Puntos de muestreo y nodos de precipitación después de la proyección realizada', fontsize=20)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'todos_nodos_precipitacion_y_localizaciones_de_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

**Limito la malla al cuadrado determinado por las coordenadas extremas de las localizaciones de muestreo.**

In [ ]:
big_radius = float(config.get('buffer', 'radius')) + 1000 # amplío el tamaño de buffer propuesto en 1000 m alrededor de cada punto
gdf_loc["buffered"] = gdf_loc.buffer(big_radius) 

# cambio la geometría por defecto de puntos a buffers
gdf_loc = gdf_loc.set_geometry("buffered")

# coordenadas extremas de los buffers
minx, miny, maxx, maxy = gdf_loc.total_bounds

print('Área de interés:')
print(f'\t(minx, miny, maxx, maxy) = ({minx}, {miny}, {maxx}, {maxy})')

# establezco de nuevo la geometría por defecto a la columna de puntos
gdf_loc = gdf_loc.set_geometry("geometry")

Genero un polígono a partir de los límites obtenidos en el punto anterior

In [ ]:
pol_geom = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
polygon = gpd.GeoDataFrame(index=[0], crs= config.get('crs', 'final_projected'), geometry=[pol_geom])  

Obtengo los puntos de la malla dentro de los límites

In [ ]:
inside = gdf_master_precip.within(polygon.geometry[0])
print(f'Número de nodos de precipitación dentro del área límite = {inside.sum()}')

In [ ]:
# filtro la malla de precipitaciones al área de interés
gdf_reduced_grid = gdf_master_precip[inside]

# reproyecto a ETRS 25830
gdf_reduced_grid = gdf_reduced_grid.to_crs(config.get('crs', 'final_projected'))
print(f'Número de nodos = {len(gdf_reduced_grid.index)}')
print(gdf_reduced_grid.head())

Genero de nuevo una gráfica con el subconjunto de nodos de precipitación y las estaciones de muestreo.

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_loc.plot(color='yellow', edgecolor='orange', marker='o', alpha=0.3, label='Ubicación de muestreo')

# gdf_reduced_grid.plot(ax=ax, marker='+')

gdf_reduced_grid.plot(ax=ax, color='red', marker='+', label='Nodo de precipitación')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='lower right', prop={'size':20}, markerscale=3)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})')
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})')
plt.title('Nodos de precipitación en área de muestreo')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'nodos_precipitacion_En_area_de_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

Zoom a las ubicaciones y su posición relativa a la malla de precipitaciones

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_reduced_grid.plot(color='red', marker='+', label='Nodo de precipitación')

# gdf_reduced_grid.plot(ax=ax, marker='+')

gdf_loc.plot(ax=ax, color='cyan', edgecolor='magenta', marker='.', alpha=0.5, label='Ubicación de muestreo')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='lower right', prop={'size':20}, markerscale=3)

ax.set_xlim(600000, 640000)
ax.set_ylim(4160000, 4180000)

plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})')
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})')
plt.title('Nodos de precipitación y puntos de muestreo')
plot_out = os.path.join(output_dir, 'zoom_distribucion_nodos_precipitacion_y_puntos_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()



In [ ]:
# Comprobación del SR del GeoDataFrame
print(gdf_reduced_grid.crs)

In [ ]:
print(f'Tamaño del DataFrame de lugares de muestreo (filas, columnas) = {gdf_loc.shape}')

In [ ]:
# Campos que forman parte del GeoDataFrame de nodos de precipitación dentro del área de estudio
print(gdf_reduced_grid.info())

In [ ]:
print(gdf_reduced_grid.head())

### 2.2. Lectura del fichero con medidas de precipitación

Leo la primera fila del fichero de precipitación. Está ubicado en el directorio *data_dir* dado por el fichero *config.ini*. El nombre del fichero es el establecido por el paquete descargado.

In [ ]:
precip_path = os.path.abspath(os.path.join(data_dir, 'pcp_red_SPAIN_1951-2020.txt'))

precipitacion = open(precip_path)
first = precipitacion.readline()
print(f'Numero de campos que contiene la primera línea = {len(first.split())}')

Leo todo el contenido del fichero. Dado que se trata de un fichero enorme (más de 3 GB), su lectura puede llevar minutos. Es por eso que si se lee por primera vez, se almacena en formato binario. Eso acelera de forma dramática las siguientes lecturas, pasando de minutos a segundos. 

Se muestra el tiempo que se ha tardado en realizar esta operación.

In [ ]:

binary_precip_path = os.path.abspath(os.path.join(data_dir, 'pcp_red_SPAIN_1951-2020.feather'))

if not os.path.exists(binary_precip_path):
    # Reading text format file
    ini_time = int(time.time())
    df_precip = pd.read_csv(precip_path, header=None, delim_whitespace=True)
    final_time = int(time.time())
    print(f'Tiempo de lectura de fichero de texto de precipitaciones = {final_time - ini_time} s')
    # saving dataframe to binary format for efficiency purposes
    # Almaceno en fichero con formato binario (tipo 'feather') el DataFrame de precipitaciones
    df_precip.columns = ['fecha'] + [str(i) for i in range(1, df_precip.shape[1])]
    ini_time = int(time.time())
    df_precip.to_feather(binary_precip_path)
    final_time = int(time.time())
    print(f'Tiempo invertido en la escritura de la información de precipitación en fichero binario = {final_time - ini_time} s')
    print(f'Fichero binario = "{binary_precip_path}"')
else:
    # Lectura del fichero binario de precipitaciones
    ini_time = int(time.time())
    df_precip = pd.read_feather(binary_precip_path)
    final_time = int(time.time())
    print(f'Tiempo invertido en la lectura del fichero de precipitaciones en formato binario = {final_time - ini_time} s')


In [ ]:
# Información sobre el fichero de precipitaciones cargado en memoria
print(df_precip.info())

In [ ]:
print(df_precip.head())

In [ ]:
print('Cabecera del fichero')
print(df_precip.columns.values)

Cambio los nombres de las columnas de este dataframe. Se supone que cada una de ellas representa la medida de uno de los nodos de precipitación dados por el fichero maestro. Ese dato es numérico.

In [ ]:
df_precip.columns = [df_precip.columns.values[0]] + [int(col_name) for col_name in df_precip.columns.values[1:]]

In [ ]:
print('Nueva cabecera del fichero')
print(df_precip.columns.values)

Proceso el campo 'fecha' del DataFrame para tratarlo como tal

In [ ]:
df_precip['fecha'] = pd.to_datetime(df_precip['fecha'], format='%Y%m%d')

In [ ]:
# la primera columna es la fecha
print(df_precip.loc[[0,1,2,3],['fecha']])

Fíjese en que ahora la fecha es un campo de typo *datetime*.

In [ ]:
df_precip.info()

### 2.3. Asociación de precipitación a cada punto de muestreo

Trabajaremos punto por punto. Es decir, **para cada localización de muestreo**, buscaremos

- los **nodos de precipitación en la zona de influencia del buffer definido alrededor de cada punto de muestreo**, y
- el **nodo de preiciptación más cercano**.

#### 2.3.1. Estimación mediante la media ponderada al inverso de la distancia a los nodos de precipitación más cercanos

In [ ]:
rad_buffer_m = float(config.get('buffer', 'radius'))
precip_date = config.get('fields', 'precip_date')
print(f'Radio de influencia de cada punto de muestreo = {rad_buffer_m} m (valor de tipo {type(rad_buffer_m)})')
measures_buffer, pairs_buffer = get_precipitations(gdf_loc, gdf_reduced_grid, df_precip, mode='buffer', \
                                                   radius_m=rad_buffer_m, field_date=precip_date)


In [ ]:
sns.set_theme(style="whitegrid")
num_nodos = [len(n.to_list()) for n in pairs_buffer['index_grid']]
sns.histplot(num_nodos)
plt.title('Nodos de precipitación asociados a cada lugar de muestreo', fontsize=22)
plt.xlabel("Número de nodos de precipitación asociados (#)", fontsize=17)
plt.ylabel("Lugares de muestreo (#)", fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'histograma_asociacion_nodos_precipitacion-puntos_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

Me hago con la columna que identifica cada estación mediante una cadena de texto 'location_label', para mostrar explícitamente información sobre el punto de muestreo. Los valores de este campo **deben ser únicos** porque se generará un fichero de salida cuyas columnas tengan los valores de este campo.

In [ ]:
key_label = config.get('fields', 'location_label')
old_column_names = list(measures_buffer.keys())
# print(old_column_names)
new_column_names = [old_column_names[0]] + gdf_loc.loc[pairs_buffer['index_loc'], key_label].to_list()

print(f"len(old_column_names) = {len(old_column_names)}, len(new_column_names) = {len(new_column_names)}")
print(f"Número de campos únicos de entrada (location_ID : {config.get('fields', 'location_ID')}) = {len(np.unique(np.array(old_column_names)))}")
print(f"Número de campos únicos de salida (location_label : {config.get('fields', 'location_label')}) = {len(np.unique(np.array(new_column_names)))}")
map_col_names = {k:v for k, v in zip(old_column_names, new_column_names)}

Generamos el DataFrame de medidas de precipitación interpolada a partir del diccionario devuelto como primer parámetro de la función *get_precipitations* en modo *buffer* (cálculo de precipitaciones usando la media ponderada del inverso de la distancia a los nodos dentro de la zona de influencia de cada punto de muestreo).

In [ ]:
df_measures_buffer = pd.DataFrame.from_dict(measures_buffer)

Renombramos las columnas a los valores del campo apuntado por el parámetro *location_label*.

In [ ]:
df_measures_buffer.rename(columns=map_col_names, inplace=True)

In [ ]:
print(f'dimensiones de measures_buffer = {df_measures_buffer.shape}')
print(df_measures_buffer.info())

Comprobamos las medidas de ia interpolación con el inverso de la distancia para el primer lugar de muestreo (el dado por el índice 0)

In [ ]:
print(f"Indice del lugar de muestreo considerado = {pairs_buffer['index_loc'][0]}")
print(f"Nodos de precipitación considerados = {pairs_buffer['index_grid'][0].to_list()}")
print(f"Distancias entre lugar de muestreo y nodos de precipitación = {pairs_buffer['distance'][0]} (en m)")

Medidas de precipitación para los nodos relacionados con el lugar de muestreo de índice 0.

In [ ]:
ind_precip = config.get('fields', 'precip_date')
indexes_grid = pairs_buffer['index_grid'][0].to_list()
print(df_precip.loc[:, [ind_precip] + indexes_grid])

Resultado de la interpolación de la precipitación en el lugar de muestreo de índice 0, ponderada por el inverso de la distancia a los nodos de precipitación con los que se relaciona.

In [ ]:
ind_precip = config.get('fields', 'precip_date')
indexes_grid = pairs_buffer['index_grid'][0].to_list()
print(df_measures_buffer.loc[:, [ind_precip, map_col_names[pairs_buffer['index_loc'][0]]]])

**Almaceno el fichero de interpolación de precipitación ponderada por la distancia en un fichero CSV**

La ruta de destino de este fichero tiene por directorio de salida *output_dir*, y por nombre de fichero *out_interp_precip*.

También se muestra el tiempo que le lleva al ordenador guardar ese fichero.

In [ ]:
# ruta del fichero de salida
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
precipitaciones_ponderado_out = os.path.join(output_dir, config.get('paths', 'out_interp_precip'))
print(f'Ruta al fichero de precipitaciones = "{precipitaciones_ponderado_out}"')

ini_time = time.time()
df_measures_buffer.to_csv(precipitaciones_ponderado_out, float_format='%.2f', index=False, header=True)
final_time = time.time()
print(f'Tiempo empleado para almacenar el fichero del histórico de precipitaciones = {int(final_time - ini_time)} s')

### 2.3. Estimo la precipitacion de cada punto de muestreo mediante asociación al nodo de precipitación más cercano

In [ ]:
rad_buffer_m = float(config.get('buffer', 'radius'))
# Atención al parámetro de la función: mode='closest'
#     define el modo de asociación de lugar de muestreo-nodo de precipitación más cercano
measures_closest, pairs_closest = get_precipitations(gdf_loc, gdf_reduced_grid, df_precip, mode='closest', \
                                     radius_m=rad_buffer_m, field_date=config.get('fields', 'precip_date'))


In [ ]:
df_muestreo_precipitacion_cercano = pd.DataFrame.from_dict(measures_closest)
print(df_muestreo_precipitacion_cercano.info())

In [ ]:
print(df_muestreo_precipitacion_cercano.head())

In [ ]:
# Reasigno el nombre a las columnas.

key_label = config.get('fields', 'location_label')
old_column_names = list(df_muestreo_precipitacion_cercano.columns.values)

new_column_names = [old_column_names[0]] + gdf_loc.loc[pairs_closest['index_loc'], key_label].to_list()
#print(len(old_column_names), np.unique(np.array(new_column_names)).size)

#print(f'len(old_column_names) = {len(old_column_names)}, len(new_column_names) = {len(new_column_names)}')
map_col_names = {k:v for k, v in zip(old_column_names, new_column_names)}

In [ ]:
df_muestreo_precipitacion_cercano.rename(columns=map_col_names, inplace=True) # future immplementation for location_label

In [ ]:
ind_precip = config.get('fields', 'precip_date')
df_muestreo_precipitacion_cercano = df_muestreo_precipitacion_cercano.set_index(ind_precip, drop=False)
print(df_muestreo_precipitacion_cercano.head())

**Y ya tengo el fichero que piden de salida para las precipitaciones**.

Ahora que están identificadas las estaciones y su nodo de malla de precipitaciones más cercanas, hay que generar el fichero de salida con formato

*DATE,location_label1,location_label2...,location_labelN*

Donde cada *location_labelx* corresponde a las medidas de precipitaciones para el lugar de muestreo indentificado con esa etiqueta.

In [ ]:
# ruta del fichero de salida
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
precipitaciones_cercanas_out = os.path.join(output_dir, config.get('paths', 'out_closest_prepip'))
print(f'Ruta al fichero de precipitaciones = "{precipitaciones_cercanas_out}"')

# Evalúo el tiempo que tarda en escribir el fichero de salida
ini_time = int(time.time())
df_muestreo_precipitacion_cercano.to_csv(precipitaciones_cercanas_out, float_format='%.2f', index=False, header=True)
final_time= int(time.time())
print(f'Tiempo de escritura del fichero "{precipitaciones_cercanas_out}" = {final_time - ini_time} s')

In [ ]:
df_related_precip_closest = pd.DataFrame.from_dict(pairs_closest)
print(df_related_precip_closest.iloc[5:10, :])

In [ ]:
print(f'Estructura del DataFrame de puntos asociados (registros, campos) = {df_related_precip_closest.shape}')

**Tengo que comprobar las asociaciones mostrando algunas de ellas en el mapa**

Determino el número de lugares de muestreo asociados a cada nodo de precipitación.

In [ ]:
res_precip_closest = df_related_precip_closest['index_grid'].value_counts()
sorted_res_precip_closest = res_precip_closest.sort_index()
#res.columns=['id','count']
print(sorted_res_precip_closest)

Diagrama de barras que muestra el número de localizaciones de muestreo asociados a cada nodo de precipitación.

In [ ]:
sorted_res_precip_closest.plot.bar()
#plt.hist(pairs['distance_m'], bins=15)
plt.title('Nodos de precipitación más cercanos a los lugares de muestreo', fontsize=20)
plt.xlabel('ID de nodo de precipitación', fontsize=17)
plt.ylabel('Número de localizaciones de muestreo relacionadas', fontsize=17)
plot_out = os.path.join(output_dir, 'localizaciones_asociadas_a_nodos_de_preciptacion_mas_cercanos.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

In [ ]:
df_1868 = df_related_precip_closest[df_related_precip_closest['index_grid'] == 1868]
print(f'Número de localizaciones asociadas al ID 1868 del nodo de precipitaciones = {len(df_1868.index)}')

In [ ]:
print(df_1868)

**1. Verificación espacial**

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_reduced_grid.plot(color='red', marker='+', markersize=50, label='Nodo de precipitación')

gdf_loc.plot(ax=ax, color='yellow', edgecolor='orange', marker='.', alpha=0.6, label='Ubicación de muestreo')

# En verde: ubicaciones de los puntos de muestreo más próximos a nodo de malla con ID=1868
gdf_loc.loc[df_1868.index].plot(ax=ax, color='green', marker='o', alpha=0.3, label='Muestreo asociado a nodo 1868')

plt.text(613000, 4163700, 'Nodo 1868', color='red', size=20)
# zoom a estas coordenadas proyectadas
ax.set_xlim(605000, 620000)
ax.set_ylim(4157500, 4170000)

cx.add_basemap(ax, crs=gdf_loc.crs.to_string(), alpha=0.5)
plt.legend(loc='upper left', prop={'size':15}, markerscale=3)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})')
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})')
plt.title('Puntos de muestreo más próximos al nodo de precipitación con ID=1868')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'localizaciones_mas_proximas_al_nodo_de_precipitacion_1868.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

**2. Verificación del valor de precipitaciones asociadas**

In [ ]:
# Primer nodo de muestreo asociado a este subconjunto
id_grid = int(df_1868.iloc[0,:].loc[['index_grid']])
id_loc = int(df_1868.iloc[0,:].loc[['index_loc']])
print(id_grid)
print(id_loc)
print(df_1868.iloc[0,:])

In [ ]:
# precipitacion asociada a este id_loc
print(df_muestreo_precipitacion_cercano.loc[:, id_loc].to_numpy())

# precipitaciones registradas en el fichero de precipitaciones
print(f'Precipitación registrada en el nodo identificado por: {id_grid}')
print(df_precip.loc[:, id_grid].to_numpy())

print('Medidas de precipitación para todos los lugares de muestreo asociados al nodo de precipitación 1868...')
for ind in df_1868.loc[:,'index_loc'].to_list():
    print(f'{ind} -> {df_muestreo_precipitacion_cercano.loc[:, ind].to_numpy()}')
    

Mostramos los nodos de la malla de precipitaciones seleccionados

In [ ]:
# Grid de nodos cercanos
plt.rcParams["figure.figsize"] = (15,12.5)

unique_grid_nodes = np.unique(df_related_precip_closest.index_grid)
print(f'Número de nodos de malla cercanos = {unique_grid_nodes.size}')

ax = gdf_loc.plot(color='yellow', edgecolor='orange', marker='o', alpha=0.3, label='puntos muestreo')
gdf_reduced_grid.loc[unique_grid_nodes].plot(ax=ax, color='red', marker='+', markersize=50, alpha=1, label='Nodo de precipitación asociado a punto de muestreo')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='upper left', prop={'size':15}, markerscale=2)
plt.title('Nodos de precipitación más próximos a los puntos de muestreo')
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})')
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'nodos_de_precipitacion_seleccionados.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

**Estadísticas sobre las distancias entre localizaciones de medida y nodos de la malla de precipitaciones**

In [ ]:
dist = df_related_precip_closest['distance'].values
message = '(media, desviación estándar, mediana, máximo, mínimo) = ({:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}) m'
print(message.format(dist.mean(), dist.std(), np.median(dist), dist.max(), dist.min()))

In [ ]:
sns.set_theme(style="whitegrid")
sns.histplot(dist, bins=20)
plt.title('Distancia (m) entre localización de medición y nodo de la malla de precipitaciones', fontsize=20)
plt.xlabel("Distancia (m)")
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'histograma_distancias_nodos_precipitacion-puntos_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

In [ ]:
# gráfica de caja y bigotes

sns.boxplot(data=df_related_precip_closest, y='distance')
plt.title('Distancia (m) entre localizaciones de muestreo y nodos de precipitación', fontsize=20)
plt.ylabel('Distancia [m]')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'boxplot_distancias_nodo_precipitacion-puntos_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

In [ ]:
# gráfica de violín

sns.violinplot(data=df_related_precip_closest, y='distance')
plt.title('Distancia (m) entre localizaciones de muestreo y nodos de precipitación', fontsize=20)
plt.ylabel('Distancia [m]')
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'violinplot_distancias_nodo_precipitacion-puntos_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

**Agrego esta información al dataframe de puntos de muestreo**

In [ ]:
print(gdf_reduced_grid)

In [ ]:
# nueva información
grid_ids = df_related_precip_closest['index_grid']
print(grid_ids)
print(len(grid_ids))
new_fields = {'ID_nodo_precip': df_related_precip_closest['index_grid'].values, 
              'dist_nodo_precip_m': df_related_precip_closest['distance'].values}
new_df = pd.DataFrame.from_dict(new_fields)
# Información y muestra de los datos a agregar al DataFrame de puntos de muestreo (df_loc)
print(new_df.info())
print(new_df.head())
print(f'\nNúmero de líneas del dataframe = {len(new_df.index)}\n')

if 'ID_nodo_precip' not in df_loc.columns:
    # agregado de nueva información al DataFrame de puntos de muestreo
    df_loc = pd.concat([df_loc, new_df], axis=1, join='inner')
    
# Verificación
print(df_loc.info())

## 3. Piezometría

La piezometría es accesible a través de una URL que apunta a un fichero alojado en la web del **Ministerio de Transición Ecológica y Reto Demográfico** (parámetro *piezometry_url* del fichero *config.ini*). Está comprimido en formato ZIP. Contiene una base de datos *MS Access* con dos pestañas:

- La primera contiene, entre otras, la ubicación de los puntos en los que se toman las medida. 
- La segunda contiene la estación piezométrica, la medida registrada y fecha.

Vamos a homogeneizar los formatos de los ficheros de entrada. Para ello, generaremos un fichero CSV por cada una de las tablas contenidas en la base de datos MS Access de la piezometría.

In [ ]:
data_dir = os.path.abspath(config.get('directories', 'data_dir'))
piezometers_path = os.path.join(data_dir, 'piezometers.csv') # CSV con información sobre los puntos de medida de piezometría
levels_path = os.path.join(data_dir, 'levels_piezometry.csv') # CSV con el resultado de las mediciones miezométricas

In [ ]:
# Descarga del fichero de datos piezométricos
piezometry_url = config.get('urls', 'piezometry_url')
prefix, name = os.path.split(piezometry_url)
piezo_path = os.path.join(data_dir, name)

piezo_res = 1
if not os.path.exists(piezo_path):
    print(f"Downloading '{piezometry_url}'. Please wait...")
    piezo_res = get_url(piezometry_url, data_dir)
else:
    print(f'"{piezo_path}" already downloaded.')

In [ ]:
df_piezometers = None
df_levels = None

# Si existen los CSV de piezometros y niveles piezometricos, los cargamos en memoria
if os.path.exists(piezometers_path):
    df_piezometers = pd.read_csv(piezometers_path)
if os.path.exists(levels_path):
    df_levels = pd.read_csv(levels_path)
    
# Si no existe uno de los dos ficheros y la descarga del ZIP de piezometría se ha 
# llevado a cabo sin problemas, descomprimimos, leemos la base de datos y 
# almacenamos cada tabla en sendos dataframes.
if (df_levels is None or df_piezometers is None) and piezo_res > 0:
    # Descompresión del fichero
    root, name = os.path.split(piezometry_url)
    piezo_path = os.path.join(data_dir, name)

    if unzip(piezo_path, data_dir) == 0:
        # Lectura del fichero de base de datos
        piezo_db_path = glob.glob(os.path.join(data_dir, '*.mdb'))
        df_piezometers, df_levels = read_db_piezometry(piezo_db_path[0])

In [ ]:
ind_piezometer = config.get('fields', 'piezo_id')
print(f'Piezometer index field = "{ind_piezometer}"')
df_piezometers = df_piezometers.set_index(ind_piezometer, drop=False)
df_levels = df_levels.set_index(ind_piezometer, drop=False)
piezo_date = config.get('fields', 'piezo_date')
print(f'Piezometer date field = "{piezo_date}"')
if piezo_date in df_levels.columns:
    print(f'Renaming piezometer date field from "{piezo_date}" to "fecha" in levels set of data.')
    df_levels.rename(columns={'FechaP':'fecha'}, inplace=True)

In [ ]:
print(df_piezometers.info())

In [ ]:
df_piezometers.head()

In [ ]:
df_levels.info()

In [ ]:
df_levels.head()

**Guardo los datos de piezometría en fichero CSV**

In [ ]:
df_piezometers.to_csv(piezometers_path, index=False)
df_levels.to_csv(levels_path, index=False)

#### Asociación entre punto de muestreo y piezómetro más próximo

Se tomará como distancia límite el campo *radius* (en metros) de la seccion *buffer* del fichero *config.ini* para que se considere esa asociación como válida.

In [ ]:
# Creo el campo de geometría para los datos de los piezómetros

# Presupongo sistema de referencia geográfico
piezo_lon = config.get('fields', 'piezo_lon_geo')
piezo_lat = config.get('fields', 'piezo_lat_geo')
piezo_crs = crs=config.get('crs', 'piezo_geographic')

if len(config.get('fields', 'piezo_lon_proj')) > 0:  
    # Si hay coordenadas proyectadas, cambio la informacion anterior
    print('Usando coordenadas proyectadas en piezometría.')
    piezo_lon = config.get('fields', 'piezo_lon_proj')
    piezo_lat = config.get('fields', 'piezo_lat_proj')
    piezo_crs = config.get('crs', 'piezo_projected')

print(f'Cooordinates fields for piezometer locations = ({piezo_lon}, {piezo_lat})')
geom = gpd.points_from_xy(df_piezometers[piezo_lon], df_piezometers[piezo_lat], crs=piezo_crs)

In [ ]:
# Genero el objeto GeoDataFrame
gdf_piezometers = gpd.GeoDataFrame(df_piezometers, geometry=geom)
ind_piezometer = config.get('fields', 'piezo_id')
print(f'Piezometers GeoDataframe index field = "{ind_piezometer}"')
gdf_piezometers = gdf_piezometers.set_index(ind_piezometer, drop=False)

Proyecto al sistema de referencia proyectado común a todas las entradas de datos.

In [ ]:
gdf_piezometers.to_crs(config.get('crs', 'final_projected'), inplace=True)

In [ ]:
print(gdf_piezometers.info())

In [ ]:
gdf_piezometers.head()

Comprobación de la unicidad del campo *piezo_label*.

In [ ]:
# Comprobación de sistema de referencia de los puntos de muestreo
print(f'SR de los puntos de muestreo = {gdf_loc.crs}')

# Y de la ubicación de los piezómetros
print(f'SR de los puntos de muestreo = {gdf_piezometers.crs}')

**Muestro ubicaciones**

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_piezometers.plot(color='red', marker='.', markersize=10, label="Piezómetros")

gdf_loc.plot(ax=ax, color='blue', alpha=0.3, marker='.', markersize=10, label="Muestreos")
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='lower right', prop={'size':20}, markerscale=3)
plt.title('Piezómetros registrados y puntos de muestreo', fontsize=20)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})', fontsize=17)
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'piezometros_registrados_y_puntos_de_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

Limito la malla al cuadrado determinado por las coordenadas extremas de las localizaciones de muestreo.

In [ ]:
big_radius = float(config.get('buffer', 'radius')) + 1000 # amplío el tamaño de buffer propuesto en 1000 m alrededor de cada punto
gdf_loc["buffered"] = gdf_loc.buffer(big_radius) 

# cambio la geometría por defecto de puntos a buffers
gdf_loc = gdf_loc.set_geometry("buffered")

# coordenadas extremas de los buffers
minx, miny, maxx, maxy = gdf_loc.total_bounds

print('Área de interés:')
print(f'\t(minx, miny, maxx, maxy) = ({minx}, {miny}, {maxx}, {maxy})')

# establezco de nuevo la geometría por defecto a la columna de puntos
gdf_loc = gdf_loc.set_geometry("geometry")


Genero un polígono a partir de los límites obtenidos en el punto anterior

In [ ]:
pol_geom = Polygon([(minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny), (minx, miny)])
polygon = gpd.GeoDataFrame(index=[0], crs= config.get('crs', 'final_projected'), geometry=[pol_geom])

In [ ]:
inside = gdf_piezometers.within(polygon.geometry[0])
print(f'Número de piezómetros en los límites de las localizaciones de muestreo = {inside.sum()}')

In [ ]:
# Zoom a la zona de muestreo
gdf_reduced_piezometers = gdf_piezometers[inside]
gdf_reduced_piezometers = gdf_reduced_piezometers.to_crs(config.get('crs', 'final_projected'))

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_reduced_piezometers.plot(color='red', marker='+', label='Ubicaciones de piezómetros')

gdf_loc.plot(ax=ax, color='yellow', edgecolor='orange', alpha=0.3, marker='.', label='Estaciones de muestreo')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='upper left', prop={'size':15}, markerscale=3)
plt.title('Piezómetros localizados en la zona de muestreo', fontsize=20)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})', fontsize=17)
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'piezometros_en_zona_de_muestreo.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

In [ ]:
df_levels.head()

In [ ]:
rad_buffer_m = float(config.get('buffer', 'radius'))
measures_piezo, related_piezo = get_piezometry(gdf_loc, gdf_reduced_piezometers, df_levels, radius_m=rad_buffer_m)

In [ ]:
df_related_piezo = pd.DataFrame.from_dict(related_piezo)

In [ ]:
print(f'Número de medidas de piezometría obtenidas = {len(measures_piezo.keys())}\n')

for i in range(len(related_piezo['index_grid'])):
    print(f"(i_loc, i_grid, distance, num_datos_piezo)-> ({related_piezo['index_loc'][i]}, {related_piezo['index_grid'][i]}, {related_piezo['distance'][i]}, {len(measures_piezo[related_piezo['index_loc'][i]])})")


Estadísticas de las distancias entre localizaciones de muestreo y piezómetros cercanos

In [ ]:
dist = df_related_piezo['distance'].values
message = '(num_datos, media, desviación estándar, mediana, máximo, mínimo) = ({}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}) m'
print(message.format(dist.size, dist.mean(), dist.std(), np.median(dist), dist.max(), dist.min()))

In [ ]:
index_closest_piezometers = np.unique(df_related_piezo.index_grid.values)
print(f'Número de piezómetros seleccionados = {len(index_closest_piezometers)}')

Imprimo las ubicaciones de esos piezometros

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_loc.plot(color='yellow', edgecolor='orange', alpha=0.1, marker='o', label='Estaciones de muestreo')
gdf_reduced_piezometers.plot(ax=ax, color='gray', marker='+', markersize=30, label='Ubicaciones de piezómetros')

gdf_reduced_piezometers.loc[index_closest_piezometers,:].plot(ax=ax, color='red', marker='+', markersize=70, label='Piezómetros seleccionados')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string())
plt.legend(loc='upper left', prop={'size':15}, markerscale=2)
plt.title('Piezómetros asociados a los puntos de muestreo', fontsize=20)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})', fontsize=17)
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'piezometros_seleccionados.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()



Veamos qué puntos con datos de piezometría son las que más frecuentemente se relacionan con los puntos de muestreo.

In [ ]:
res = df_related_piezo['index_grid'].value_counts()
sorted_res = res.sort_index()
#res.columns=['id','count']
print(sorted_res)

In [ ]:
sorted_res.plot.bar()
#plt.hist(pairs['distance_m'], bins=15)
plt.title('Estaciones piezométricas más usadas', fontsize=20)
plt.xlabel('IDPIEZ', fontsize=17)
plt.ylabel('Localizaciones de muestreo', fontsize=17)
plot_out = os.path.join(output_dir, 'localizaciones_asociadas_a_piezometros.jpg')
plt.savefig(plot_out, dpi=300)
plt.show()

In [ ]:
gdf_reduced_piezometers.loc[[2015]]

Muestro las localizaciones de muestreo asociadas a la estación peizométrica más relacionada (index_grid = 2015)

In [ ]:
plt.rcParams["figure.figsize"] = (15,12.5)

ax = gdf_loc.plot(color='gray', edgecolor='gray', alpha=0.1, marker='o', label='Otras estaciones de muestreo')

selected_muestreo = df_related_piezo['index_grid'] == 2015
indexes = np.unique(df_related_piezo[selected_muestreo]['index_loc'].values)

gdf_loc.loc[indexes,:].plot(ax=ax, color='magenta', alpha=0.1, marker='o', label='Estaciones de muestreo 2015')
cx.add_basemap(ax, crs=gdf_loc.crs.to_string(), alpha=0.5)
gdf_reduced_piezometers[gdf_reduced_piezometers.index != 2015].plot(ax=ax, color='orange', marker='+', markersize=70, label='Otros piezómetros seleccionados')
gdf_reduced_piezometers.loc[[2015]].plot(ax=ax, color='blue', marker='+', markersize=50, label='Piezómetro con ID = 2015')

plt.legend(loc='upper left', prop={'size':15}, markerscale=2)
plt.title('Localizaciones asociadas al piezómetro con ID=2015', fontsize=20)
plt.xlabel(f'Longitud ({config.get("crs", "final_projected")})', fontsize=17)
plt.ylabel(f'Latitud ({config.get("crs", "final_projected")})', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'localizaciones_asociadas_piezometro_2015.jpg')
plt.savefig(plot_out, dpi=300)
plt.show()

Hago algo de estadística con las distancias: histograma, diagrama de caja y bigotes y diagrama de violín. Se trata de visualizar la distribución de distancias entre los puntos de muestro y las ubicaciones de las estaciones piezométricas.


In [ ]:
sns.set_theme(style="whitegrid")
sns.histplot(df_related_piezo['distance'].values, bins=20)
plt.title('Distancia (m) entre localización de los puntos de muestreo y los piezómetros', fontsize=20)
plt.xlabel("Distancia (m)", fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'histograma_distancias_a_piezometros.jpg')
plt.savefig(plot_out, dpi=300)
plt.show()

In [ ]:
# gráfica de caja y bigote
sns.set_theme(style="whitegrid")
sns.boxplot(data=df_related_piezo, y='distance')
plt.title('Distancia entre puntos de muestreo y ubicaciones de piezómetros', fontsize=20)
plt.ylabel('Distancia [m]', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'boxplot_distancias_a_piezometros.jpg')

plt.savefig(plot_out, dpi=300)

plt.show()

In [ ]:
# gráfica de violín

sns.set_theme(style="whitegrid")

sns.violinplot(data=df_related_piezo, y='distance')
plt.title('Distancia entre puntos de muestreo y ubicaciones de piezómetros', fontsize=20)
plt.ylabel('Distancia [m]', fontsize=17)
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
plot_out = os.path.join(output_dir, 'violinplot_distancias_a_piezometros.jpg')
plt.savefig(plot_out, dpi=300)

plt.show()

In [ ]:
# Vuelco el contenido del dataframe de correspondencias entre localizaciones de muestreo y piezómetros a un fichero
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
related_piezo_path = os.path.join(output_dir, 'correspondencias_piezometros.csv')
df_related_piezo.to_csv(related_piezo_path, index=False)

**Fichero de salida para las medidas piezométricas**

In [ ]:
print(measures_piezo.keys())

In [ ]:
measures_piezo[0]

In [ ]:

for k, v in measures_piezo.items():
    print(f'{k} -> {v}')
    break

In [ ]:
for k, v in measures_piezo.items():
    print(f"{k}:{gdf_loc.loc[k, config.get('fields', 'location_label')]}")
    break

In [ ]:
piezo_field = config.get('fields', 'piezo_measure')
for k, v in measures_piezo.items():
    # v es un dataframe con columnas 'fecha' y 'Cota_NP_msnm'. Cambio el nombre de la columna 'Cota_NP_msnm' al ID del punto de muestreo
    # temp = v.rename(columns={piezo_field:int(k)})
    temp = v.rename(columns={piezo_field:gdf_loc.loc[k, config.get('fields', 'location_label')]})
    
print(temp)

In [ ]:
# Fichero de salida para las medidas piezométricas

df_data_piezo = None
piezo_field = config.get('fields', 'piezo_measure')
print(f'Piezometry measure field = "{piezo_field}"')
for k, v in measures_piezo.items():
    # v es un dataframe con columnas 'fecha' y 'Cota_NP_msnm'. Cambio el nombre de la columna 'Cota_NP_msnm' al ID del punto de muestreo
    # temp = v.rename(columns={piezo_field:int(k)})
    temp = v.rename(columns={piezo_field:gdf_loc.loc[k, config.get('fields', 'location_label')]})

    if df_data_piezo is None:
        df_data_piezo = temp
    else:
        df_data_piezo = df_data_piezo.merge(temp, left_on='fecha', right_on='fecha', how='outer')

print(df_data_piezo.info())

In [ ]:
df_data_piezo.head()

In [ ]:
output_dir = os.path.abspath(config.get('directories', 'output_dir'))
piezometrias_out = os.path.join(output_dir, config.get('paths', 'out_longer_piezo'))
# tiempo de escritura de fichero
ini_time = int(time.time())
df_data_piezo.to_csv(piezometrias_out, index=False, header=True)
final_time = int(time.time())
print(f'Tiempo de escritura del fichero "{piezometrias_out}" = {final_time - ini_time} s')

In [ ]:
# Primeros 50 resultados.
print(df_data_piezo.head(50))

In [ ]:
# Últimos 50 resultados
print(df_data_piezo.tail(50))